In [1]:
import os
from PIL import Image
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
from utils import datasets
from utils import utils
from utils import train_val
from utils import net #网络文件于此
from utils import metrics
import warnings

# 完全禁用警告
warnings.filterwarnings("ignore")

In [2]:
from utils.utils import Config,Logs,BestSelector
config=utils.Config(
    dataset_sep=[
        0.82,0.17,0.01          
        ],
    resize_size=168,
    batch_size=128,
    lr=0.001,
    epochs=30,
    hidden_size=256,
    optim="Adam",
    momentum=0.9,
    weight_decay=1e-4,
    seed=42,
    mean= [0.50638 ,0.49962538 ,0.45205265],
    std=[0.23568255 ,0.24141274 ,0.25167742],
    checkpoint_interval=0.25,#只保存4个模型
    source_dir=r"Cifar-10",#原始数据集，每个分类一个文件夹，每个文件夹里包含多个图片
    data_path=r"data\Cifar-10",#项目数据集
    # classes=["Apple","Carambola","Pear","Plum","Tomatoes"],
    device="cuda" if torch.cuda.is_available() else "cpu",
)
print(config)

dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 168 
batch_size : 128 
lr : 0.001 
epochs : 30 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
checkpoint_interval : 0.25 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 


In [3]:
from tqdm import tqdm

#设置随机数种子
torch.manual_seed(config.seed)

isSplit= (not datasets.check_data_exist(config.data_path))#数据集不存在，则从原始数据存放处，转移数据集
if isSplit:
    print(f"{config.data_path} 数据集不存在，将从source_dir:{config.source_dir}中获取数据")
    print(f"清理源文件夹:{config.data_path}")
    datasets.clear_folder(config.data_path)
    for dir in tqdm(config.classes,desc="处理原始数据："):
        source_dir=os.path.join(config.source_dir, dir)
        print(source_dir)
        datasets.split_data(source_dir,target_dir=config.data_path,label=dir,sep=config.dataset_sep)
else:
    print(f"{config.data_path} 数据集已存在，无需重新划分")



data\Cifar-10 数据集已存在，无需重新划分


In [4]:
'''
def get_data_path(typ,path=config.data_path):
    return os.path.join(path,typ)

train_dataset=datasets.CustomImageDataset(
    get_data_path("train"),
    classes=config.classes,
    transform=datasets.get_transform(
        resize_size=config.resize_size,
        mean=config.mean,std=config.std
        )
)

val_dataset=datasets.CustomImageDataset(
    get_data_path("val"),
    classes=config.classes,
    transform=datasets.get_transform(
        "val",
        config.resize_size,
        
        mean=config.mean,
        std=config.std
        )
)
'''


'\ndef get_data_path(typ,path=config.data_path):\n    return os.path.join(path,typ)\n\ntrain_dataset=datasets.CustomImageDataset(\n    get_data_path("train"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        resize_size=config.resize_size,\n        mean=config.mean,std=config.std\n        )\n)\n\nval_dataset=datasets.CustomImageDataset(\n    get_data_path("val"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        "val",\n        config.resize_size,\n        \n        mean=config.mean,\n        std=config.std\n        )\n)\n'

#### 加载数据集，可本地数据集。调用datasets.CustomImageDataset

In [5]:
#加载cifar-10
import torchvision
train_val_dataset = torchvision.datasets.CIFAR10(
    root='./data/Cifar-10', 
    train=True, 
    download=True, 
    transform=datasets.get_transform(
        chance="train",
        resize_size=config.resize_size,
        mean=(0.5, 0.5, 0.5), 
        std=(0.5, 0.5, 0.5)
    )
)
train_dataset,val_dataset=datasets.get_parts_of_datasets(train_val_dataset,rate=0.75,only_train=False)#训练验证3-1开

test_dataset = torchvision.datasets.CIFAR10(root='./data/Cifar-10', train=False, download=True, transform=datasets.get_transform(
    chance="val",
    resize_size=config.resize_size,
    mean=(0.5, 0.5, 0.5), 
    std=(0.5, 0.5, 0.5)
))
print(f"原始训练集大小：{len(train_dataset)}")
print(f"原始验证集大小：{len(val_dataset)}")
print(f"原始测试集大小：{len(test_dataset)}")

Files already downloaded and verified
Files already downloaded and verified
原始训练集大小：37500
原始验证集大小：12500
原始测试集大小：10000


#### load数据集，并根据数据量进行裁剪

In [6]:
crop_rate=0.25
train_dataset_crop=datasets.get_parts_of_datasets(train_dataset,crop_rate)
val_dataset_crop=datasets.get_parts_of_datasets(val_dataset,crop_rate)
test_dataset_crop=datasets.get_parts_of_datasets(test_dataset,crop_rate)
train_dataset_size=len(train_dataset_crop)
val_dataset_size=len(val_dataset_crop)
test_dataset_size=len(test_dataset_crop)

print(f"本次训练用训练集大小：{len(train_dataset_crop)}")
print(f"本次训练用验证集大小：{len(val_dataset_crop)}")
print(f"本次测试用测试集大小：{len(test_dataset_crop)}")

train_loader=DataLoader(train_dataset_crop,batch_size=config.batch_size,shuffle=True,drop_last=True)
val_loader=DataLoader(val_dataset_crop,batch_size=config.batch_size,shuffle=False)
test_loader=DataLoader(test_dataset_crop,batch_size=config.batch_size,shuffle=False)

print(f"tarin_dataloader加载完毕, {len(train_loader)}个batch, batch大小为{config.batch_size}")
print(f"val_dataloader  加载完毕, {len(val_loader)}个batch, batch大小为{config.batch_size}")
print(f"test_dataloader 加载完毕, {len(test_loader)}个batch, batch大小为{config.batch_size}")


本次训练用训练集大小：9375
本次训练用验证集大小：3125
本次测试用测试集大小：2500
tarin_dataloader加载完毕, 73个batch, batch大小为128
val_dataloader  加载完毕, 25个batch, batch大小为128
test_dataloader 加载完毕, 20个batch, batch大小为128


#### 记录本次训练和测试用的数据量，还有数据样本信息

In [7]:
#查看数据加载情况
for inputs, labels in train_loader:
    inputs=inputs.to(config.device)
    labels=labels.to(config.device)
    print(inputs.shape)
    print(labels.shape)
    config.update(
        inputs_shape=inputs.shape
        )
    break
config.update(
    train_datasize=train_dataset_size,
    val_datasetsize=val_dataset_size,
    test_datasetsize=test_dataset_size,
    datasets_crop_rate=crop_rate,
    classes=train_val_dataset.classes,#原始数据集保留classes
)
config

torch.Size([128, 3, 168, 168])
torch.Size([128])


dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 168 
batch_size : 128 
lr : 0.001 
epochs : 30 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
checkpoint_interval : 0.25 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 
inputs_shape : torch.Size([128, 3, 168, 168]) 
train_datasize : 9375 
val_datasetsize : 3125 
test_datasetsize : 2500 
datasets_crop_rate : 0.25 
classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 

#### 加载预训练模型

In [8]:

def get_pretrained(hidden_size=config.hidden_size,config=config):
    '''
        获取预训练模型
        @param hidden_size: 隐藏层大小
        @param config: 配置文件
        @return: 预训练模型
    '''
    # model=net.BinaryClassificationMobileNetV3Large(out_size=len(config.classes))
    # model=net.AutoCNN(
    #     input_channels=config.inputs_shape[1],
    #     input_size=config.inputs_shape[2:],
    #     num_classes=len(config.classes),
    #     mlp_hidden_size=256
    # )
    model=net.ResNet(
        num_classes=len(config.classes),
        input_channels=config.inputs_shape[1],
    )#最后全局池化层压下了尺寸，不需要提供输入尺寸
    return model.to(config.device)

test_model=get_pretrained()
measurer=metrics.ModelMeasurer(test_model)
unit=1
inference_time=measurer.get_inference_time(input_shape=config.inputs_shape,unit=unit)
parameters_num=measurer.get_parameters_num()
print(f"inference_time:{inference_time} s")
print(f"parameters_num:{parameters_num}")
def seconds_to_hms(seconds):
    hours = seconds // 3600
    remaining_seconds = seconds % 3600
    minutes = remaining_seconds // 60
    seconds = remaining_seconds % 60
    return f"{int(hours)}: {int(minutes)}: {int(seconds)}"
timestamp=(config.epochs*inference_time*(len(train_loader)+len(val_loader)))/unit
print(f"训练完成预计用时:{seconds_to_hms(timestamp)}")
config.update(
    network=test_model.__class__.__name__,
    inference_time=inference_time,
    parameters_num=parameters_num,
)
config

Testing ...: 100%|██████████| 300/300 [00:13<00:00, 22.44it/s]

inference_time:0.044010154024759926 ms
parameters_num:11186890
训练完成预计用时:0: 2: 9


dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 168 
batch_size : 128 
lr : 0.001 
epochs : 30 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
checkpoint_interval : 0.25 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 
inputs_shape : torch.Size([128, 3, 168, 168]) 
train_datasize : 9375 
val_datasetsize : 3125 
test_datasetsize : 2500 
datasets_crop_rate : 0.25 
classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 
network : ResNet 
inference_time : 0.044010154024759926 
parameters_num : 11186890 

In [ ]:

bestMod=utils.BestSelector(acc=0)
train_logs=utils.Logs()
model=get_pretrained()
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
if config.optim == "Adam":
    optimizer = optim.Adam(model.parameters(), lr=config.lr)
else:
    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=1e-4)
    
bestMod,train_logs=train_val.train_model(
            model, 
            criterion,
            optimizer, 
            train_loader,
            val_loader,
            bestMod=bestMod,
            train_logs=train_logs,
            config=config, 
            checkpoint_interval=config.checkpoint_interval,
            show_progress_interval=3
            # num_epochs=config.epochs
        )



Training Epochs:   3%|▎         | 1/30 [00:25<12:16, 25.40s/it]

Epoch 1/30, Loss: 2.0277,  Val Accuracy: 0.2957, AP: 0.1140, Precision: 0.3263, Recall: 0.2974
当前最好的模型： acc : 0.29568 ,loss : 2.027743342804582 ,precision : 0.32625946039380027 ,recall : 0.29742494079799986 ,ap : 0.11401514356778034 ,epoch : 0 


Training Epochs:  13%|█▎        | 4/30 [01:41<11:00, 25.42s/it]

Epoch 4/30, Loss: 1.6171,  Val Accuracy: 0.4346, AP: 0.1014, Precision: 0.4411, Recall: 0.4336
当前最好的模型： acc : 0.43456 ,loss : 1.617102327412122 ,precision : 0.4411379356109234 ,recall : 0.43355927148345214 ,ap : 0.10137807600654461 ,epoch : 3 


Training Epochs:  23%|██▎       | 7/30 [02:55<09:28, 24.71s/it]

...

Training Epochs:  63%|██████▎   | 19/30 [07:42<04:22, 23.87s/it]

Epoch 19/30, Loss: 0.9643,  Val Accuracy: 0.6122, AP: 0.1034, Precision: 0.6326, Recall: 0.6111
当前最好的模型： acc : 0.61216 ,loss : 0.9642606973648071 ,precision : 0.6325863379014207 ,recall : 0.6110934920023355 ,ap : 0.10342420197954032 ,epoch : 18 


Training Epochs:  73%|███████▎  | 22/30 [08:56<03:16, 24.52s/it]

Epoch 22/30, Loss: 0.8839,  Val Accuracy: 0.5734, AP: 0.1078, Precision: 0.6173, Recall: 0.5739
当前最好的模型： acc : 0.6176 ,loss : 0.9192486487022818 ,precision : 0.6208336630865773 ,recall : 0.6177364211505211 ,ap : 0.09889777522983192 ,epoch : 20 


Training Epochs:  80%|████████  | 24/30 [09:48<02:31, 25.26s/it]

In [10]:
print(f"{config.epochs} epoch中 最好的模型")
print(bestMod)

1 epoch中 最好的模型
acc : 0.31072 ,loss : 2.011701871271003 ,precision : 0.2972016861629805 ,recall : 0.3088747080907871 ,ap : 0.12449937806569095 ,epoch : 0 


#### 保存模型超参数和训练日志

In [ ]:
saveDir=r'save_weights'
saveDir=os.path.join(
        saveDir,
        f'{bestMod.model.__class__.__name__[:10]}-acc={round(bestMod.acc,5)}-loss={round(bestMod.loss,3)}-max_epochs={config.epochs}'
)
utils.saveProcess(
    saveDir=saveDir,
    bestMod=bestMod,
    train_log=train_logs,
    config=config
)

save_weights\ResNet-acc=0.31072-loss=2.012-max_epochs=1
{'acc': 0.31072, 'model': 'save_weights\\ResNet-acc=0.31072-loss=2.012-max_epochs=1\\best.pth', 'loss': 2.011701871271003, 'precision': 0.2972016861629805, 'recall': 0.3088747080907871, 'ap': 0.12449937806569095, 'epoch': 0, 'checkpoints': {'checkpoint_0': 'save_weights\\ResNet-acc=0.31072-loss=2.012-max_epochs=1\\checkpoint_0.pth'}}


#### 进行测试

In [ ]:
import torch
from utils import utils,train_val
import os

# config=Config(os.path.join(dir,'config.json'))
# model=BestSelector(os.path.join(dir,'metrics.json'))
# saveDir=r'save_weights\BinaryClassificationMobileNetV3Large-acc=0.74336-loss=1.671334-max_epochs=40-1100'

Model,config,logs=utils.loadProcess(saveDir=saveDir)
metrics=train_val.validate_model(
    model=Model.model,
    val_loader=test_loader,
    device=config.device,
    only_val=True
)

metrics

模型测试中:: 100%|██████████| 20/20 [00:00<00:00, 30.16it/s]


{'Accuracy': 0.33,
 'Precision': 0.343839494985118,
 'Recall': 0.327905984972946,
 'F1': 0.2971122184198728,
 'AP': 0.13231780506062604}

#### 保存数据到tensorboard

In [13]:
from utils import metrics,utils


recoder=metrics.TensorboardRecorder(
    log_dir="runs/"
)

recoder.logs_scalars(
    logs.logs,
    prefix="train"
)



#### 多个模型的训练日志展示
 

In [14]:
from utils import metrics,utils
import os
model_dirs=[
    r"D:\Desktop\深度学习\12.康复训练\My_DL_Utils\Project\save_weights\AutoCNN-acc=0.3832-loss=2.082528-max_epochs=10-1100",
    r"D:\Desktop\深度学习\12.康复训练\My_DL_Utils\Project\save_weights\BinaryClassificationMobileNetV3Large-acc=0.662-loss=1.719393-max_epochs=10-1100"
]
recorders=[]
for model_dir in model_dirs:
    Model,config,logs=utils.loadProcess(model_dir)
    dirname=os.path.basename(model_dir)
    recorder=metrics.TensorboardRecorder(log_dir=f"runs/{dirname}")
    recorders.append(recorder)
    recorder.logs_scalars(
        scalars=logs.logs,
        prefix="train"
    )



FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Desktop\\深度学习\\12.康复训练\\My_DL_Utils\\Project\\save_weights\\AutoCNN-acc=0.3832-loss=2.082528-max_epochs=10-1100\\metrics.json'